In [1]:
import seaborn as sns
import pandas as pd
import numpy as np

#### Lien du dataset
https://www.kaggle.com/cityofLA/la-restaurant-market-health-data

### Import des 2 datasets de Restaurant Market Health

In [2]:
df_inspections = pd.read_csv("csv_files/restaurant-and-market-health-inspections.csv")

df_inspections.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67573 entries, 0 to 67572
Data columns (total 20 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   serial_number        67573 non-null  object
 1   activity_date        67573 non-null  object
 2   facility_name        67573 non-null  object
 3   score                67573 non-null  int64 
 4   grade                67570 non-null  object
 5   service_code         67573 non-null  int64 
 6   service_description  67573 non-null  object
 7   employee_id          67573 non-null  object
 8   facility_address     67573 non-null  object
 9   facility_city        67573 non-null  object
 10  facility_id          67573 non-null  object
 11  facility_state       67573 non-null  object
 12  facility_zip         67573 non-null  object
 13  owner_id             67573 non-null  object
 14  owner_name           67573 non-null  object
 15  pe_description       67573 non-null  object
 16  prog

In [4]:
df_inspections.head(10)

,serial_number,activity_date,facility_name,score,grade,service_code,service_description,employee_id,facility_address,facility_city,facility_id,facility_state,facility_zip,owner_id,owner_name,pe_description,program_element_pe,program_name,program_status,record_id
0,DAJ00E07B,2017-12-29T00:00:00.000,HABITAT COFFEE SHOP,95,A,1,ROUTINE INSPECTION,EE0000923,3708 N EAGLE ROCK BLVD,LOS ANGELES,FA0170465,CA,90065,OW0178123,GLASSELL COFFEE SHOP LLC,RESTAURANT (0-30) SEATS MODERATE RISK,1631,HABITAT COFFEE SHOP,ACTIVE,PR0160774
1,DAQOKRFZB,2017-12-29T00:00:00.000,REILLY'S,92,A,1,ROUTINE INSPECTION,EE0000633,100 WORLD WAY # 120,LOS ANGELES,FA0244690,CA,90045,OW0208441,"AREAS SKYVIEW LAX JV, LLC",RESTAURANT (0-30) SEATS MODERATE RISK,1631,REILLY'S,ACTIVE,PR0193026
2,DASJI4LUR,2017-12-29T00:00:00.000,STREET CHURROS,93,A,1,ROUTINE INSPECTION,EE0000835,6801 HOLLYWOOD BLVD # 253,LOS ANGELES,FA0224109,CA,90028,OW0228670,"STREETCHURROS, INC",RESTAURANT (0-30) SEATS LOW RISK,1630,STREET CHURROS,ACTIVE,PR0179282
3,DA40LU5AT,2017-12-29T00:00:00.000,TRINITI ECHO PARK,94,A,1,ROUTINE INSPECTION,EE0000923,1814 W SUNSET BLVD,LOS ANGELES,FA0252528,CA,90026-3227,OW0246287,AMERICAN HOSPITALITY,RESTAURANT (0-30) SEATS MODERATE RISK,1631,TRINITI ECHO PARK,ACTIVE,PR0201836
4,DAXV2RMYC,2017-12-29T00:00:00.000,POLLEN,94,A,1,ROUTINE INSPECTION,EE0000923,2100 ECHO PARK AVE,LOS ANGELES,FA0252789,CA,90026,OW0246479,"POLLEN AND PASTRY , LLC",RESTAURANT (0-30) SEATS MODERATE RISK,1631,POLLEN,ACTIVE,PR0202148
5,DAMV56BMJ,2017-12-29T00:00:00.000,THE SPOT GRILL,93,A,1,ROUTINE INSPECTION,EE0000727,10004 NATIONAL BLVD,LOS ANGELES,FA0245224,CA,90034,OW0240313,LIQUOR CLUB INC,RESTAURANT (0-30) SEATS HIGH RISK,1632,THE SPOT GRILL,INACTIVE,PR0193589
6,DAKSJB0AB,2017-12-29T00:00:00.000,7 ELEVEN #37215A,95,A,1,ROUTINE INSPECTION,EE0000839,3330 W FLORENCE AVE,LOS ANGELES,FA0243779,CA,90043-4706,OW0239028,"S & U INVESTMENTS, INC.","FOOD MKT RETAIL (2,000+ SF) LOW RISK",1613,7 ELEVEN #37215A,ACTIVE,PR0192029
7,DAQOJLNCD,2017-12-29T00:00:00.000,CPK,91,A,1,ROUTINE INSPECTION,EE0000633,100 WORLD WAY # 126,LOS ANGELES,FA0225926,CA,90045,OW0208441,"AREAS SKYVIEW LAX JV, LLC",RESTAURANT (61-150) SEATS HIGH RISK,1638,CPK,ACTIVE,PR0181243
8,DAACRFWTS,2017-12-29T00:00:00.000,PHO LALA,95,A,1,ROUTINE INSPECTION,EE0000699,3500 W 6TH ST STE 226,LOS ANGELES,FA0248964,CA,90020,OW0243289,"VANILLA #, INC",RESTAURANT (0-30) SEATS HIGH RISK,1632,PHO LALA,ACTIVE,PR0197670
9,DAE1JXAWW,2017-12-29T00:00:00.000,ABC DONUTS,91,A,1,ROUTINE INSPECTION,EE0000923,3027 N SAN FERNANDO RD UNIT 103,LOS ANGELES,FA0002761,CA,90065,OW0026313,"NOUNG, SAING HOW",RESTAURANT (0-30) SEATS MODERATE RISK,1631,ABC DONUTS,ACTIVE,PR0045001


In [3]:
df_violations = pd.read_csv("csv_files/restaurant-and-market-health-violations.csv")

df_violations.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 313675 entries, 0 to 313674
Data columns (total 25 columns):
 #   Column                 Non-Null Count   Dtype 
---  ------                 --------------   ----- 
 0   serial_number          313675 non-null  object
 1   activity_date          313675 non-null  object
 2   facility_name          313675 non-null  object
 3   violation_code         313675 non-null  object
 4   violation_description  313675 non-null  object
 5   violation_status       313675 non-null  object
 6   points                 313675 non-null  int64 
 7   grade                  313618 non-null  object
 8   facility_address       313675 non-null  object
 9   facility_city          313675 non-null  object
 10  facility_id            313675 non-null  object
 11  facility_state         313675 non-null  object
 12  facility_zip           313675 non-null  object
 13  employee_id            313675 non-null  object
 14  owner_id               313675 non-null  object
 15  

In [5]:
df_violations.head(10)

,serial_number,activity_date,facility_name,violation_code,violation_description,violation_status,points,grade,facility_address,facility_city,...,owner_name,pe_description,program_element_pe,program_name,program_status,record_id,score,service_code,service_description,row_id
0,DA08R0TCU,2018-03-30T00:00:00.000,KRUANG TEDD,F030,# 30. Food properly stored; food storage conta...,OUT OF COMPLIANCE,1,A,5151 HOLLYWOOD BLVD,LOS ANGELES,...,5151 HOLLYWOOD LLC,RESTAURANT (31-60) SEATS HIGH RISK,1635,KRUANG TEDD,ACTIVE,PR0031205,92,1,ROUTINE INSPECTION,DA08R0TCUF030
1,DA08R0TCU,2018-03-30T00:00:00.000,KRUANG TEDD,F027,# 27. Food separated and protected,OUT OF COMPLIANCE,1,A,5151 HOLLYWOOD BLVD,LOS ANGELES,...,5151 HOLLYWOOD LLC,RESTAURANT (31-60) SEATS HIGH RISK,1635,KRUANG TEDD,ACTIVE,PR0031205,92,1,ROUTINE INSPECTION,DA08R0TCUF027
2,DA08R0TCU,2018-03-30T00:00:00.000,KRUANG TEDD,F035,# 35. Equipment/Utensils - approved; installed...,OUT OF COMPLIANCE,1,A,5151 HOLLYWOOD BLVD,LOS ANGELES,...,5151 HOLLYWOOD LLC,RESTAURANT (31-60) SEATS HIGH RISK,1635,KRUANG TEDD,ACTIVE,PR0031205,92,1,ROUTINE INSPECTION,DA08R0TCUF035
3,DA08R0TCU,2018-03-30T00:00:00.000,KRUANG TEDD,F033,# 33. Nonfood-contact surfaces clean and in go...,OUT OF COMPLIANCE,1,A,5151 HOLLYWOOD BLVD,LOS ANGELES,...,5151 HOLLYWOOD LLC,RESTAURANT (31-60) SEATS HIGH RISK,1635,KRUANG TEDD,ACTIVE,PR0031205,92,1,ROUTINE INSPECTION,DA08R0TCUF033
4,DA08R0TCU,2018-03-30T00:00:00.000,KRUANG TEDD,F029,"# 29. Toxic substances properly identified, st...",OUT OF COMPLIANCE,1,A,5151 HOLLYWOOD BLVD,LOS ANGELES,...,5151 HOLLYWOOD LLC,RESTAURANT (31-60) SEATS HIGH RISK,1635,KRUANG TEDD,ACTIVE,PR0031205,92,1,ROUTINE INSPECTION,DA08R0TCUF029
5,DA08R0TCU,2018-03-30T00:00:00.000,KRUANG TEDD,F044,"# 44. Floors, walls and ceilings: properly bui...",OUT OF COMPLIANCE,1,A,5151 HOLLYWOOD BLVD,LOS ANGELES,...,5151 HOLLYWOOD LLC,RESTAURANT (31-60) SEATS HIGH RISK,1635,KRUANG TEDD,ACTIVE,PR0031205,92,1,ROUTINE INSPECTION,DA08R0TCUF044
6,DA08R0TCU,2018-03-30T00:00:00.000,KRUANG TEDD,F006,# 06. Adequate handwashing facilities supplied...,OUT OF COMPLIANCE,2,A,5151 HOLLYWOOD BLVD,LOS ANGELES,...,5151 HOLLYWOOD LLC,RESTAURANT (31-60) SEATS HIGH RISK,1635,KRUANG TEDD,ACTIVE,PR0031205,92,1,ROUTINE INSPECTION,DA08R0TCUF006
7,DA0GBKW3F,2018-03-30T00:00:00.000,SPROUTS FARMERS MARKET#403,F044,"# 44. Floors, walls and ceilings: properly bui...",OUT OF COMPLIANCE,1,A,2245 YOSEMITE DR,LOS ANGELES,...,"SPROUTS, INC.","FOOD MKT RETAIL (25-1,999 SF) HIGH RISK",1612,MEAT MARKET,ACTIVE,PR0179289,97,1,ROUTINE INSPECTION,DA0GBKW3FF044
8,DA0GBKW3F,2018-03-30T00:00:00.000,SPROUTS FARMERS MARKET#403,F039,# 39. Wiping cloths: properly used and stored,OUT OF COMPLIANCE,1,A,2245 YOSEMITE DR,LOS ANGELES,...,"SPROUTS, INC.","FOOD MKT RETAIL (25-1,999 SF) HIGH RISK",1612,MEAT MARKET,ACTIVE,PR0179289,97,1,ROUTINE INSPECTION,DA0GBKW3FF039
9,DA0GBKW3F,2018-03-30T00:00:00.000,SPROUTS FARMERS MARKET#403,F037,# 37. Adequate ventilation and lighting; desig...,OUT OF COMPLIANCE,1,A,2245 YOSEMITE DR,LOS ANGELES,...,"SPROUTS, INC.","FOOD MKT RETAIL (25-1,999 SF) HIGH RISK",1612,MEAT MARKET,ACTIVE,PR0179289,97,1,ROUTINE INSPECTION,DA0GBKW3FF037
